In [115]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import os
from datetime import datetime, timedelta
from pathlib import Path

# Reading data
只能在"昱嘉"和"允謙"資料夾執行

fun: read_final_data() -> tr_X, tr_Y, test_X, test_Y

In [38]:
def read_final_data() -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    cwd = os.getcwd()
    data_path = Path(cwd).parent

    train_X = pd.read_csv(data_path / "data" /  "train.csv")
    train_Y = pd.read_csv(data_path / "data" /  "train_label.csv")
    test_X = pd.read_csv(data_path / "data" /  "test.csv")
    test_Y = pd.read_csv(data_path / "data" /  "test_nolabel.csv")
    return train_X, train_Y, test_X, test_Y

In [30]:
train_X, train_Y, test_X, test_Y = read_final_data()

## adr 是負數的情況

服務不周所以賠錢了

In [15]:
# train_data_X[train_data_X["adr"] < 0]

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,2015-07-01
63,63,Resort Hotel,0,2,2015,July,27,1,0,1,...,No Deposit,240.0,NaN,0,Transient,-0.729340,0,1,Check-Out,2015-07-02
64,64,City Hotel,0,6,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,-2.902196,0,0,Check-Out,2015-07-03
119,119,Resort Hotel,0,737,2015,July,27,1,0,0,...,No Deposit,NaN,NaN,0,Transient,-88.642804,0,0,Check-Out,2015-07-01
243,243,Resort Hotel,1,368,2015,July,27,3,3,7,...,No Deposit,40.0,NaN,0,Contract,-4.441218,0,0,Canceled,2015-05-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91283,91283,City Hotel,1,629,2017,March,13,30,0,2,...,Non Refund,1.0,NaN,0,Transient,-27.102503,0,0,Canceled,2015-10-21
91305,91305,City Hotel,1,629,2017,March,13,30,0,2,...,Non Refund,1.0,NaN,0,Transient,-44.775831,0,0,Canceled,2015-10-21
91500,91500,Resort Hotel,0,23,2017,March,13,31,0,2,...,No Deposit,NaN,NaN,0,Transient,-14.504251,0,1,Check-Out,2017-04-02
91503,91503,Resort Hotel,0,71,2017,March,13,31,0,2,...,No Deposit,NaN,NaN,0,Transient,-12.919657,0,0,Check-Out,2017-04-02


# First try

先拿掉一些比較細節的columns, 然後做average把每一個row變成每一天的觀察值

In [39]:
train_X.columns

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [53]:
X = train_X.drop(["ID", "lead_time", "arrival_date_week_number", "country", "market_segment", "distribution_channel",
              "is_repeated_guest", "previous_cancellations", "previous_bookings_not_canceled", "reserved_room_type",
             "assigned_room_type", "booking_changes", "deposit_type", "agent", "company", "days_in_waiting_list",
             "customer_type", "required_car_parking_spaces", "total_of_special_requests"], axis=1)

In [36]:
def create_datetime(year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
    return pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")

In [54]:
year = X["arrival_date_year"]
month = X["arrival_date_month"]
date = X["arrival_date_day_of_month"]

X["date"] = create_datetime(year, month, date)
X.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], axis=1, inplace = True)

In [56]:
X.head()

,hotel,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,adr,reservation_status,reservation_status_date,date
0,Resort Hotel,0,0,0,2,0.0,0,BB,-6.305161,Check-Out,2015-07-01,2015-07-01
1,City Hotel,0,0,2,1,0.0,0,HB,75.052227,Check-Out,2015-07-03,2015-07-01
2,City Hotel,0,0,2,2,0.0,0,HB,74.546401,Check-Out,2015-07-03,2015-07-01
3,City Hotel,0,0,2,2,0.0,0,HB,76.376288,Check-Out,2015-07-03,2015-07-01
4,City Hotel,0,0,2,2,0.0,0,HB,49.411647,Check-Out,2015-07-03,2015-07-01


In [86]:
X[X["is_canceled"] == 1].head()

,hotel,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,adr,reservation_status,reservation_status_date,date
23,City Hotel,1,0,2,2,0.0,0,HB,68.863895,Canceled,2015-06-16,2015-07-01
24,City Hotel,1,0,2,2,0.0,0,HB,70.077704,Canceled,2015-06-16,2015-07-01
25,City Hotel,1,0,2,2,0.0,0,HB,82.360331,Canceled,2015-06-16,2015-07-01
26,City Hotel,1,0,2,2,0.0,0,HB,77.250048,Canceled,2015-06-16,2015-07-01
27,City Hotel,1,0,2,2,0.0,0,HB,78.099031,Canceled,2015-06-16,2015-07-01


In [84]:
X["reservation_status_date"] = pd.to_datetime(X["reservation_status_date"], format="%Y-%m-%d")

In [88]:
(X["date"] - X["reservation_status_date"]).mean()

Timedelta('26 days 20:24:01.557505')

In [99]:
df = X[X["is_canceled"] != 1]

In [100]:
df = df.drop(["is_canceled", "reservation_status", "reservation_status_date"], axis=1)

In [101]:
df

,hotel,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,adr,date
0,Resort Hotel,0,0,2,0.0,0,BB,-6.305161,2015-07-01
1,City Hotel,0,2,1,0.0,0,HB,75.052227,2015-07-01
2,City Hotel,0,2,2,0.0,0,HB,74.546401,2015-07-01
3,City Hotel,0,2,2,0.0,0,HB,76.376288,2015-07-01
4,City Hotel,0,2,2,0.0,0,HB,49.411647,2015-07-01
...,...,...,...,...,...,...,...,...,...
91525,Resort Hotel,3,7,2,0.0,0,HB,79.453407,2017-03-31
91527,Resort Hotel,0,2,2,0.0,0,BB,-6.822102,2017-03-31
91528,Resort Hotel,0,1,2,0.0,0,BB,90.814554,2017-03-31
91529,Resort Hotel,3,7,2,0.0,0,BB,38.135565,2017-03-31
